In [ ]:
import numpy as np
import xarray as xr
import matplotlib as mpl
import matplotlib.pyplot as plt
import cmocean as cmo
import pandas as pd
import matplotlib.animation as animation
from matplotlib.colors import LightSource
from matplotlib import cm
from matplotlib.lines import Line2D
from matplotlib.gridspec import GridSpec

In [ ]:
# This notebook uses the full datasets (output frequency of .125 yr) instead of the compressed datasets (output frequency of 10 yr)
# For access to the full datasets or any related inquiries, please contact Franka Jesse at t.f.jesse@uu.nl
 
data_dir = '/Volumes/T7/P1_DIR/Model_output'

exp_mod_modt = ['P1_MEDIUM_laddie_NOcalving_rhoice_concat', 'P1_MEDIUM_QUAD_NOcalving', 'P1_MEDIUM_PICO_NOcalving_C023', 'P1_MEDIUM_PLUME_NOcalving']
exp_mod_higt = ['P1_MEDIUM_laddie_NOcalving_rhoice_concat', 'P1_MEDIUM_QUAD_NOcalving_hightune', 'P1_MEDIUM_PICO_NOcalving_C023_hightune', 'P1_MEDIUM_PLUME_NOcalving_hightune']
exp_hig_higt = ['P1_WARM_laddie_NOcalving_rhoice_concat', 'P1_WARM_QUAD_NOcalving', 'P1_WARM_PICO_NOcalving_C023', 'P1_WARM_PLUME_NOcalving']
exp_hig_modt = ['P1_WARM_laddie_NOcalving_rhoice_concat', 'P1_WARM_QUAD_NOcalving_modtune', 'P1_WARM_PICO_NOcalving_C023_modtune', 'P1_WARM_PLUME_NOcalving_modtune']

exp_mod_modt_calving = ['P1_MEDIUM_laddie_rhoice_concat', 'P1_MEDIUM_QUAD', 'P1_MEDIUM_PICO_C023', 'P1_MEDIUM_PLUME']
exp_hig_modt_calving = ['P1_WARM_laddie_rhoice_concat', 'P1_WARM_QUAD_modtune', 'P1_WARM_PICO_C023_modtune', 'P1_WARM_PLUME_modtune']

In [ ]:
lws=[3,2,2,2]
labels = ['LA', 'QU', 'PI', 'PL',]
colors = ['C3', 'C9', 'C8', 'C7']
alphs = [.7,1,.7,1 ]
ls = ['solid', 'dashed', 'solid', 'dashed']
panel_labels = ['(a)', '(b)', '(c)','(d)', '(e)', '(f)','(g)', '(h)',]

In [ ]:
# Plot VAF loss rate, VAF, and GL retreat rate

fig = plt.figure(figsize=(8,12), dpi=300)
gs = GridSpec(4, 2, width_ratios=[1, 1], height_ratios=[1.5,1,1,1])

ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1], sharex=ax1, sharey=ax1)
ax3 = fig.add_subplot(gs[2], sharex=ax1)
ax4 = fig.add_subplot(gs[3], sharex=ax1, sharey=ax3)
ax5 = fig.add_subplot(gs[4], sharex=ax1)
ax6 = fig.add_subplot(gs[5], sharex=ax1, sharey=ax5)

ax_VAFs         = [ax3, ax3, ax4, ax4]
ax_VAFlosss     = [ax1, ax1, ax2, ax2]
ax_GLrates      = [ax5, ax5, ax6, ax6]


# PLOT PERTURBATION EXPERIMENTS
for j, exp_sets in enumerate([exp_mod_modt, exp_mod_modt_calving, exp_hig_modt,exp_hig_modt_calving]):
    ax_VAF      = ax_VAFs[j]
    ax_VAFloss  = ax_VAFlosss[j]
    ax_GLrate   = ax_GLrates[j]

    for i, exp in enumerate(exp_sets):

        # Read VAF data
        ds_plot = xr.open_dataset(f'{data_dir}/../PostProcess/VAF_{exp}.nc')
        # Compute VAF loss
        N = 200     # smoothing window = 200 * dt = 25 year
        VAF_smooth  = np.convolve(ds_plot.VAF, np.ones(N)/N, mode='valid')
        VAF_loss    = np.diff(-1*VAF_smooth)*8

        # GL retreat rate
        ds_GL = xr.open_dataset(f'{data_dir}/../PostProcess/GL_{exp}.nc')
        smooth_window   = 50  # yr
        dts_per_year    = 8   # dt = .125 yr
        slice = int(smooth_window*dts_per_year) # = 400 dts
        GL_smooth = np.convolve(ds_GL.GLpos, np.ones(N)/N, mode='valid')
        GL_smooth_diff = np.diff(-1*GL_smooth)*8

        # Integrated BMB
        ds_BMB = xr.open_dataset(f'{data_dir}/../PostProcess/BMB_{exp}.nc')
        smooth_window = 50  # yr
        dts_per_year = 8    # dt = .125 yr
        slice = int(smooth_window*dts_per_year)

        # Plot VAF
        ax_VAF.plot(ds_plot.time, ds_plot.VAF, label=labels[i], lw=lws[i], c=colors[i], alpha=alphs[j], ls=ls[j])
        # Set labels
        ax_VAF.set_xticks([0,200,400,600,800,1000])
        plt.setp(ax_VAF.get_xticklabels(), visible=False)
        ax_VAF.set_xlim(0,1000)
        ax_VAF.set_ylim(1e4,3.5e4)
        if j ==5:
            pass
        else:
            # Plot VAF loss
            ax_VAFloss.plot(ds_plot.time.values[0::slice][:-1]+25, -np.diff(ds_plot.VAF.values[0::slice])/(smooth_window), label=labels[i], lw=lws[i], c=colors[i],alpha=alphs[j], ls=ls[j])
            #ax_VAFloss.scatter(ds_plot.time.values[0::slice][:-1]+25, -np.diff(ds_plot.VAF.values[0::slice])/(smooth_window),  c=colors[i])

            # Set labels
            ax_VAFloss.set_xticks([0,200,400,600,800,1000])
            plt.setp(ax_VAFloss.get_xticklabels(), visible=False)
            ax_VAFloss.set_ylim(-1,50)

            # Plot GL retreat rate
            ax_GLrate.plot(ds_GL.time.values[0::slice][:-1]+25, -np.diff(ds_GL.GLpos.values[0::slice])/(smooth_window),label=labels[i], lw=lws[i], c=colors[i],alpha=alphs[j], ls=ls[j])
            #ax_GLrate.scatter(ds_GL.time.values[0::slice][:-1]+25, -np.diff(ds_GL.GLpos.values[0::slice])/(smooth_window), c=colors[i])
            
            # Set labels
            ax_GLrate.set_xticks([0,200,400,600,800,1000])
            ax_GLrate.set_ylim(-100,1000)

            # # Set labels
            ax_GLrate.set_xlabel('Model year')
            ax_GLrate.set_xticks([0,200,400,600,800,1000])




# ax_VAFloss.scatter(ds_control.time.values[0::slice][:-1]+25, -np.diff(ds_control.VAF.values[0::slice])/(smooth_window),  c='grey')

for i, axs in enumerate([ax1, ax2, ax3, ax4, ax5, ax6,]):
    axs.grid(True, alpha=0.2, ls='dashed')
    axs.vlines(200, -100000, 900000, zorder=1, color='k', lw=2, alpha=0.8)
    axs.vlines(1000, -100000, 900000, zorder=1, color='k', lw=4, alpha=0.8)
    axs.text(0.02, 0.02, panel_labels[i], transform=axs.transAxes,
            fontsize=10, fontweight='bold', va='bottom', ha='left')

for axs in [ax2, ax4, ax6]:
    plt.setp(axs.get_yticklabels(), visible=False)

ax1.set_title('Moderate-melt')
ax2.set_title('High-melt')

ax3.set_ylabel(r'VAF [Gt]')
ax1.set_ylabel(r'VAF loss [Gt/yr]')
ax5.set_ylabel(r'GL retreat rate [m/yr]')


lineLA = Line2D([0], [0], lw=3, label='LADDIE', color='C3')
lineQU = Line2D([0], [0], lw=2, label='Quadratic', color='C9')
linePI = Line2D([0], [0], lw=2, label='PICO', color='C8')
linePL = Line2D([0], [0], lw=2, label='Plume', color='C7')
lineNO = Line2D([0], [0], lw=2, label='Default', color='k')
lineca = Line2D([0], [0], lw=2, ls='dashed', label='Calving', color='k')


legend = ax2.legend(handles=[lineLA, lineQU, linePI, linePL, lineNO, lineca])
i=0
for text, handle in zip(legend.get_texts(), [lineLA, lineQU, linePI, linePL]):
    text.set_color(handle.get_color())
    text.set_fontweight("bold")         # Make label bold

fig.align_ylabels() 

fig.savefig('figE1.pdf', dpi=300, bbox_inches='tight')